In [1]:
from skimage import io, transform
import os
import numpy as np
#from 人脸标准化 import normalization
import sklearn.preprocessing


In [2]:
# 数据集地址  最后的斜杆一定不要漏
path = 'D:/QIN/Face-classfication/lfw_database/lfw/'
# 处理后的数据集
path_normalization = 'D:/QIN/Face-classfication/lfw_database/normalization_lfw/'
# 模型保存地址
model_path = './人脸识别/model.ckpt'
# tfrecord文件存放路径
TFRECORD_FILE = "./人脸识别/tfrecords/"

#全局one-hot编码空间
label_binarizer = ""

#train test
# 处理后的数据集
path_normalization_train = 'D:/QIN/Face-classfication/lfw_database/normalization_lfw/train/'

path_normalization_test = 'D:/QIN/Face-classfication/lfw_database/normalization_lfw/test/'

In [3]:
def read_img(path):
    map_path, map_relative = [path +x for x in os.listdir(path) if os.path.isfile(path + x) ], [y for y in os.listdir(path)]
    return map_path, map_relative

In [4]:
read_img(path_normalization)

(['D:/QIN/Face-classfication/lfw_database/normalization_lfw/Aaron_Eckhart_0001.jpg',
  'D:/QIN/Face-classfication/lfw_database/normalization_lfw/Aaron_Guiel_0001.jpg',
  'D:/QIN/Face-classfication/lfw_database/normalization_lfw/Aaron_Patterson_0001.jpg',
  'D:/QIN/Face-classfication/lfw_database/normalization_lfw/Aaron_Peirsol_0001.jpg',
  'D:/QIN/Face-classfication/lfw_database/normalization_lfw/Aaron_Peirsol_0002.jpg',
  'D:/QIN/Face-classfication/lfw_database/normalization_lfw/Aaron_Peirsol_0003.jpg',
  'D:/QIN/Face-classfication/lfw_database/normalization_lfw/Aaron_Peirsol_0004.jpg',
  'D:/QIN/Face-classfication/lfw_database/normalization_lfw/Aaron_Pena_0001.jpg',
  'D:/QIN/Face-classfication/lfw_database/normalization_lfw/Aaron_Sorkin_0001.jpg',
  'D:/QIN/Face-classfication/lfw_database/normalization_lfw/Aaron_Sorkin_0002.jpg',
  'D:/QIN/Face-classfication/lfw_database/normalization_lfw/Aaron_Tippin_0001.jpg',
  'D:/QIN/Face-classfication/lfw_database/normalization_lfw/Abbas_Kiaro

In [5]:
#默认按列归一化
def def_normalization(x,h = 1):
    # 记录归一化全局最大值最小值，回代时需要
    if h==1:
        #按列归一化
        amin, amax = np.min(x, 0), np.max(x, 0)
        xx = (x - amin) / (amax - amin)
    else:
        #按行归一化
        amin, amax = np.min(x, 1), np.max(x, 1)
        xx = ((x.T - amin) / (amax - amin)).T
    #记录归一化最大值最小值，回代时需要
    return xx

#使用one-hot编码，将离散特征的取值扩展到了欧式空间
def def_one_hot(x):
    if label_binarizer == "":
        binarizer = sklearn.preprocessing.LabelBinarizer()
    else:
        binarizer = label_binarizer
    binarizer.fit(x)
    y= binarizer.transform(x)
    return y
    

In [6]:
# 读取处理后的图片，顺序打乱，划分测试和训练
def read_new_img(path):
    map_path, map_relative = read_img(path)
    imgs=[]
    labels=[]
    for idx, folder, in enumerate(map_path):
        if map_relative[idx].split(".")[0] == 'train' or map_relative[idx].split(".")[0] == 'test':
            continue
        img = io.imread(path_normalization+map_relative[idx])
        img = transform.resize(img, (50, 50))
        imgs.append(img)
        if map_relative[idx].split(".")[0].split("_")[1].isdigit():
            la_name = map_relative[idx].split(".")[0].split("_")[0]
            labels.append(la_name)
        else:
            la_name = map_relative[idx].split(".")[0].split("_")[0] + '_'+map_relative[idx].split(".")[0].split("_")[1]
            labels.append(la_name)
    x_data, x_label = np.array(imgs), np.array(def_one_hot(np.array(labels)))
    data = []
    sigle_data = []
    for i in range(len(x_data)):
        sigle_data.append(x_data[i])
        sigle_data.append(x_label[i])
        data.append(sigle_data)
        sigle_data = []
        # 打乱顺序
    data = np.array(data)
    num_example = data.shape[0]
    np.random.shuffle(data)
    # 将所有数据分为训练集和验证集
    ratio = 0.8
    img_train = []
    label_train = []
    img_test = []
    label_test = []
    for i in range(int(ratio*num_example)):
        img_train.append(data[i][0])
        label_train.append(data[i][1])
    img_train = np.array(img_train)
    label_train = np.array(label_train)
    for i in range(int(ratio*num_example),num_example):
        img_test.append(data[i][0])
        label_test.append(data[i][1])
    img_test = np.array(img_test)
    label_test = np.array(label_test)
    print(img_train)
    return img_train,img_test,label_train,label_test

In [7]:

read_new_img(path_normalization)

D:\Python\anaconda3.4\envs\py36\lib\site-packages\skimage\transform\_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


[[[[6.23529412e-01 3.17647059e-01 2.66666667e-01]
   [6.47058824e-01 3.49019608e-01 2.86274510e-01]
   [6.98039216e-01 4.03921569e-01 3.29411765e-01]
   ...
   [3.60784314e-01 1.33333333e-01 7.84313725e-02]
   [1.72549020e-01 7.45098039e-02 5.88235294e-02]
   [1.29411765e-01 3.13725490e-02 1.56862745e-02]]

  [[6.23529412e-01 3.25490196e-01 2.70588235e-01]
   [6.54901961e-01 3.56862745e-01 2.94117647e-01]
   [6.98039216e-01 4.03921569e-01 3.29411765e-01]
   ...
   [4.47058824e-01 2.19607843e-01 1.64705882e-01]
   [1.96078431e-01 8.62745098e-02 7.05882353e-02]
   [1.33333333e-01 2.35294118e-02 7.84313725e-03]]

  [[6.31372549e-01 3.33333333e-01 2.78431373e-01]
   [6.66666667e-01 3.68627451e-01 3.05882353e-01]
   [6.98039216e-01 4.11764706e-01 3.37254902e-01]
   ...
   [6.39215686e-01 4.15686275e-01 3.49019608e-01]
   [2.82352941e-01 1.60784314e-01 1.41176471e-01]
   [2.03921569e-01 8.23529412e-02 6.27450980e-02]]

  ...

  [[4.54901961e-01 2.11764706e-01 1.13725490e-01]
   [4.62745098e-

(array([[[[6.23529412e-01, 3.17647059e-01, 2.66666667e-01],
          [6.47058824e-01, 3.49019608e-01, 2.86274510e-01],
          [6.98039216e-01, 4.03921569e-01, 3.29411765e-01],
          ...,
          [3.60784314e-01, 1.33333333e-01, 7.84313725e-02],
          [1.72549020e-01, 7.45098039e-02, 5.88235294e-02],
          [1.29411765e-01, 3.13725490e-02, 1.56862745e-02]],
 
         [[6.23529412e-01, 3.25490196e-01, 2.70588235e-01],
          [6.54901961e-01, 3.56862745e-01, 2.94117647e-01],
          [6.98039216e-01, 4.03921569e-01, 3.29411765e-01],
          ...,
          [4.47058824e-01, 2.19607843e-01, 1.64705882e-01],
          [1.96078431e-01, 8.62745098e-02, 7.05882353e-02],
          [1.33333333e-01, 2.35294118e-02, 7.84313725e-03]],
 
         [[6.31372549e-01, 3.33333333e-01, 2.78431373e-01],
          [6.66666667e-01, 3.68627451e-01, 3.05882353e-01],
          [6.98039216e-01, 4.11764706e-01, 3.37254902e-01],
          ...,
          [6.39215686e-01, 4.15686275e-01, 3.4901

In [8]:
#以下是网络的信息
import tensorflow as tf
#初始化权值
def weight_variable(shape,name):
    initial = tf.truncated_normal(shape,stddev=0.01)#生成一个截断的正态分布
    return tf.Variable(initial,name=name)

#初始化偏置
def bias_variable(shape,name):
    initial = tf.constant(0.01,shape=shape)
    return tf.Variable(initial,name=name)

#卷积层
def conv2d(x,W):
    #x input tensor of shape `[batch, in_height, in_width, in_channels]`
    #W filter / kernel tensor of shape [filter_height, filter_width, in_channels, out_channels]
    #`strides[0] = strides[3] = 1`. strides[1]代表x方向的步长，strides[2]代表y方向的步长
    #padding: A `string` from: `"SAME", "VALID"`
    return tf.nn.conv2d(x,W,strides=[1,1,1,1],padding='SAME')

#池化层
def max_pool_2x2(x):
    #ksize [1,x,y,1]
    return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')

D:\Python\anaconda3.4\envs\py36\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [9]:
def xunlianlo(path):
    # read_dispose_img(path)
    x_train, x_test, y_train, y_test = read_new_img(path_normalization)
    m,n = y_train.shape
    print("m:")
    print(m)
    print("n:")
    print(n)
    # 命名空间
    # 定义两个placeholder
    x = tf.placeholder(tf.float32, [None, 50, 50, 3], name='x-input')
    y = tf.placeholder(tf.float32, [None, n], name='y-input')
    # 改变x的格式转为4D的向量[batch, in_height, in_width, in_channels]`
    x_image = tf.reshape(x, [-1, 50, 50, 3], name='x_image')

    # 初始化第一个卷积层的权值和偏置
    W_conv1 = weight_variable([5, 5, 3, 32], name='W_conv1')  # 5*5的采样窗口，32个卷积核从3个平面抽取特征
    b_conv1 = bias_variable([32], name='b_conv1')  # 每一个卷积核一个偏置值

    # 把x_image和权值向量进行卷积，再加上偏置值，然后应用于relu激活函数
    conv2d_1 = conv2d(x_image, W_conv1) + b_conv1
    h_conv1 = tf.nn.leaky_relu(conv2d_1)
    h_pool1 = max_pool_2x2(h_conv1)  # 进行max-pooling

    # 初始化第二个卷积层的权值和偏置
    W_conv2 = weight_variable([5, 5, 32, 64], name='W_conv2')  # 5*5的采样窗口，64个卷积核从32个平面抽取特征
    b_conv2 = bias_variable([64], name='b_conv2')  # 每一个卷积核一个偏置值

    # 把h_pool1和权值向量进行卷积，再加上偏置值，然后应用于relu激活函数

    conv2d_2 = conv2d(h_pool1, W_conv2) + b_conv2
    h_conv2 = tf.nn.leaky_relu(conv2d_2)
    h_pool2 = max_pool_2x2(h_conv2)  # 进行max-pooling

    # 300*300的图片第一次卷积后还是300*300，第一次池化后变为150*150
    # 第二次卷积后为150*150，第二次池化后变为了75*75
    # 进过上面操作后得到64张7*7的平面

    # 初始化第一个全连接层的权值
    W_fc1 = weight_variable([13 * 13 * 64, 1024], name='W_fc1')  # 上一场有75*75*64个神经元，全连接层有1024个神经元
    b_fc1 = bias_variable([1024], name='b_fc1')  # 1024个节点

    # 把池化层2的输出扁平化为1维
    h_pool2_flat = tf.reshape(h_pool2, [-1, 13 * 13 * 64], name='h_pool2_flat')

    # 求第一个全连接层的输出
    wx_plus_b1 = tf.matmul(h_pool2_flat, W_fc1) + b_fc1
    h_fc1 = tf.nn.leaky_relu(wx_plus_b1)

    # keep_prob用来表示神经元的输出概率
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob, name='h_fc1_drop')

    # 初始化第二个全连接层
    W_fc2 = weight_variable([1024, n], name='W_fc2')
    b_fc2 = bias_variable([n], name='b_fc2')
    wx_plus_b2 = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

    # 计算输出
    prediction = tf.nn.leaky_relu(wx_plus_b2)

    tf.add_to_collection('predictions', prediction)

    p = tf.nn.softmax(wx_plus_b2)
    tf.add_to_collection('p', p)
    # 交叉熵代价函数
    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=prediction),
                                   name='cross_entropy')

    # 使用AdamOptimizer进行优化
    train_step = tf.train.AdamOptimizer(0.0001).minimize(cross_entropy)
    # train_step = tf.train.GradientDescentOptimizer(5).minimize(cross_entropy)

    # 求准确率
    # 结果存放在一个布尔列表中
    correct_prediction = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))  # argmax返回一维张量中最大的值所在的位置
    # 求准确率
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

    #保存模型使用环境
    saver = tf.train.Saver()

    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())

        # 创建一个协调器，管理线程
        coord = tf.train.Coordinator()
        # 启动QueueRunner, 此时文件名队列已经进队
        threads = tf.train.start_queue_runners(sess=sess, coord=coord)

        for i in range(1):
            # 训练模型
            sess.run(train_step, feed_dict={x: x_train, y: y_train, keep_prob: 0.9})

            test_acc = sess.run(accuracy, feed_dict={x: x_test, y: y_test, keep_prob: 1.0})
            train_acc = sess.run(accuracy, feed_dict={x: x_train, y: y_train, keep_prob: 1.0})
            print("训练第 " + str(i) + " 次, 训练集准确率= " + str(train_acc) + " , 测试集准确率= " + str(test_acc))

            if test_acc == 1 and train_acc >= 0.95:
                print("准确率完爆了")
                # 保存模型
                saver.save(sess, 'nn/my_net.ckpt')
                break

        # 通知其他线程关闭
        coord.request_stop()
        # 其他所有线程关闭之后，这一函数才能返回
        coord.join(threads)

In [ ]:

xunlianlo(path_normalization)

D:\Python\anaconda3.4\envs\py36\lib\site-packages\skimage\transform\_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


[[[[8.11411765e-01 6.98823529e-01 4.34823529e-01]
   [8.34705882e-01 7.29882353e-01 4.15411765e-01]
   [8.46352941e-01 7.53176471e-01 3.72705882e-01]
   ...
   [1.94117647e-01 1.12588235e-01 9.31764706e-02]
   [1.70823529e-01 1.12588235e-01 3.88235294e-02]
   [1.63058824e-01 1.04823529e-01 3.10588235e-02]]

  [[8.38627451e-01 7.21098039e-01 4.39215686e-01]
   [8.50745098e-01 7.41058824e-01 4.12000000e-01]
   [8.51098039e-01 7.53176471e-01 3.57450980e-01]
   ...
   [2.22705882e-01 1.32745098e-01 1.09333333e-01]
   [2.33411765e-01 1.70784314e-01 1.07686275e-01]
   [2.29372549e-01 1.66745098e-01 1.03647059e-01]]

  [[8.65294118e-01 7.40196078e-01 4.39215686e-01]
   [8.54705882e-01 7.37450980e-01 3.85686275e-01]
   [8.47254902e-01 7.41764706e-01 3.30784314e-01]
   ...
   [2.75686275e-01 1.89215686e-01 1.47058824e-01]
   [2.57647059e-01 1.91176471e-01 1.28431373e-01]
   [2.50000000e-01 1.83529412e-01 1.20784314e-01]]

  ...

  [[8.23333333e-01 7.41372549e-01 2.74313725e-01]
   [8.22941176e-